# 05_compute_ISC

In [1]:
import numpy as np
from os.path import join as pjoin
from os.path import isdir
import os
import mne_bids
import mne
from mne_bids import write_raw_bids, BIDSPath
from scipy import stats
import re
from scipy import signal
import pandas as pd
from scipy import signal, fftpack
import pickle

In [2]:
# define variables
sub_list = ['{0:0>2d}'.format(sub) for sub in np.arange(1,12)]
run_list = ['{0:0>2d}'.format(run) for run in np.arange(1,9)]
band_names = ['delta', 'theta', 'alpha', 'beta', 'gamma']

# set path
pre_root = '/nfs/e5/studyforrest/forrest_movie_meg/gump_meg_bids'
post_root = pjoin(pre_root, 'derivatives', 'preproc_meg-mne_mri-fmriprep')

results_pth = '/nfs/e5/studyforrest/forrest_movie_meg/tech_val_results/'
if os.path.exists(results_pth) is False:
    os.mkdir(results_pth)

In [3]:
def extract_megdata(bids_root, sub, run):

    sub_path = BIDSPath(subject=sub, run=run, task='movie', session='movie', root=bids_root)
    raw = mne_bids.read_raw_bids(sub_path)
    
    return raw

In [4]:
def extract_events(bids_root, sub, run):
    
    raw = extract_megdata(bids_root, sub, run)
    events = mne.find_events(raw, stim_channel='UPPT001', min_duration=2/raw.info['sfreq'])
    
    return events

In [5]:
def get_picks(data):
    # get valid channels
    ch_name_picks = mne.pick_channels_regexp(data.ch_names, regexp='M[LRZ]...-4503')
    type_picks = mne.pick_types(data.info, meg=True)
    picks= np.intersect1d(ch_name_picks, type_picks)
    return picks

In [6]:
def band_split(raw, band_name):
    """

    """
    band_freq = {'delta': [1, 4],
                 'theta': [4, 8],
                 'alpha': [8, 13],
                 'beta': [13, 30],
                 'gamma': [30, 100]}
    
    raw_band = raw.copy().load_data().filter(l_freq=band_freq[band_name][0], h_freq=band_freq[band_name][1])

    return raw_band

In [7]:
def compute_isc(data1, data2):

    """calculate inter-subject correlation along the determined axis.

    Parameters
    ----------

        data1, data2: array,
            shape = [n_samples, n_features].

    Returns
    -------
        isc: point-to-point functional connectivity list of
            data1 and data2, shape = [n_samples].

    """

    data1 = np.nan_to_num(data1)
    data2 = np.nan_to_num(data2)

    z_data1 = np.nan_to_num(stats.zscore(data1, axis=-1))
    z_data2 = np.nan_to_num(stats.zscore(data2, axis=-1))
    corr = np.sum(z_data1*z_data2, axis=-1)/(np.size(data1, -1))

    return corr

In [8]:
def get_power_data(data_sub, events_sub, band_names, picks_ref):

    picks_sub = get_picks(data_sub)
    # match channels
    picks = np.intersect1d(picks_sub, picks_ref)
    
    # mark bad channels
    bad_idx = []
    if len(picks) != len(picks_ref):
        bad_picks = np.union1d(np.setdiff1d(picks_sub, picks_ref), np.setdiff1d(picks_ref, picks_sub))
        for chn in bad_picks:
            bad_idx.append(np.where(picks == chn)[0][0])
    
    power_data = {}
    for band_name in band_names:
        # band_split
        band_sub = band_split(data_sub, band_name=band_name)
        band_sub_data = band_sub.get_data(picks=picks)
        # hilbert xfm
        envlope = np.abs(signal.hilbert(band_sub_data))
        if len(bad_idx) != 0:
            for idx in bad_idx:
                envlope[idx,:] = 0
        # downsampling
        envlope_dsamp = envlope.take(events_sub[1:-1:25,0], axis=1)
        power_data[band_name] = envlope_dsamp
    
    del data_sub
    
    return power_data 

In [ ]:
# compute power data
# get ref picks
data_ref = extract_megdata(pre_root, '01', '01')
picks_ref = get_picks(data_ref)
del data_ref

power_data_dir = pjoin(results_pth, 'band_power')
pre_powerdata_dir = pjoin(power_data_dir, 'pre')
post_powerdata_dir = pjoin(power_data_dir, 'post')
if os.path.exists(power_data_dir) is False:
    os.mkdir(power_data_dir)
if os.path.exists(pre_powerdata_dir) is False:
    os.mkdir(pre_powerdata_dir)
if os.path.exists(post_powerdata_dir) is False:
    os.mkdir(post_powerdata_dir)
    
for sub in sub_list:
    if sub == '01':
        run_ls = run_list + ['09']
    else:
        run_ls = run_list
    
    for run in run_ls:      
        megdata = extract_megdata(pre_root, sub, run)
        events = extract_events(pre_root, sub, run)
        pre_power_data = get_power_data(megdata, events, band_names, picks_ref)
        with open(pjoin(pre_powerdata_dir, 'sub-{0}_run-{1}.pickle'.format(sub, run)), 'wb') as fp:
            pickle.dump(pre_power_data, fp)  
            
        megdata = extract_megdata(post_root, sub, run)
        events = extract_events(post_root, sub, run)
        post_power_data = get_power_data(megdata, events, band_names, picks_ref)
        
        with open(pjoin(post_powerdata_dir, 'sub-{0}_run-{1}.pickle'.format(sub, run)), 'wb') as fp:
            pickle.dump(post_power_data, fp)  

In [13]:
def compute_bandISC(power_data_dir, sub_list, run_list, band_names):
    
    bandISC = {band_name : np.zeros((len(run_list), len(sub_list), len(sub_list), len(picks_ref)))
               for band_name in band_names}
    
    for run in run_list:
        if int(run) >= 7:
            sub_ls = sub_list[1:]
        else:
            sub_ls = sub_list

        # pair-wise ISC
        for i, sub1 in enumerate(sub_ls):
            with open(pjoin(power_data_dir, 'sub-{0}_run-{1}.pickle'.format(sub1, run)), 'rb') as fp:
                powerdata_sub1 = pickle.load(fp) 

            for sub2 in sub_ls[i+1:]:
                with open(pjoin(power_data_dir, 'sub-{0}_run-{1}.pickle'.format(sub2, run)), 'rb') as fp:
                    powerdata_sub2 = pickle.load(fp) 

                for band_name in band_names:
                    isc = compute_isc(powerdata_sub1[band_name], powerdata_sub2[band_name])
                    bandISC[band_name][int(run)-1, int(sub1)-1, int(sub2)-1] = np.asarray(isc)

    return bandISC

In [14]:
# pre
bandISC_pre = compute_bandISC(pre_powerdata_dir, sub_list, run_list, band_names)
with open(pjoin(results_pth, 'bandISC_pre.pickle'), 'wb') as fp:
    pickle.dump(bandISC_pre, fp)  

# post
bandISC_post = compute_bandISC(post_powerdata_dir, sub_list, run_list, band_names)
with open(pjoin(results_pth, 'bandISC_post.pickle'), 'wb') as fp:
    pickle.dump(bandISC_post, fp)  